![NYC Skyline](img/nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many [Airbnb](https://www.airbnb.com/) listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this notebook, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

Our goals are to convert untidy data into appropriate formats to analyze, and answer key questions including:

- What is the average price, per night, of an Airbnb listing in NYC?
- How does the average price of an Airbnb listing, per month, compare to the private rental market?
- How many adverts are for private rooms?
- How do Airbnb listing prices compare across the five NYC boroughs?


In [1]:
# Dependencies
import numpy as np
import pandas as pd

In [2]:
# Load in datasets
prices = pd.read_csv('data/airbnb_price.csv')

xls = pd.ExcelFile('data/airbnb_room_type.xlsx')

room_types = xls.parse(0)

reviews = pd.read_csv('data/airbnb_last_review.tsv', sep='\t')

In [3]:
# Preview the prices df
prices.head()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [4]:
# Remove whitespace and string characters trailing the numeric values in price column
prices['price'] = prices['price'].str.replace(" dollars", "")

# Convert price column to numeric data type
prices['price'] = pd.to_numeric(prices['price'])

In [5]:
# Get a description of prices df
prices.describe()

,listing_id,price
count,2.520900e+04,25209.000000
mean,2.068922e+07,141.777936
std,1.102928e+07,147.349137
min,2.595000e+03,0.000000
25%,1.202273e+07,69.000000
50%,2.234391e+07,105.000000
75%,3.037669e+07,175.000000
max,3.645581e+07,7500.000000


In [6]:
# Remove outliers 
free_listing = prices['price'] == 0

prices = prices.loc[~free_listing]

In [7]:
# Calculate the average price
avg_price = round(prices['price'].mean(), 2)

# Display the results
avg_price

141.82

In [8]:
# Add a new column price_per_month to prices dataframe
prices['price_per_month'] = (prices['price'] * 365) / 12

# Find the average price of the price_per_month column
average_price_per_month = round(prices['price_per_month'].mean(), 2)

# Display the results
average_price_per_month

4313.61

In [9]:
# Calculate the difference between an Airbnb listing vs. private market
difference = round((average_price_per_month - 3100), 2)

# Display results
difference

1213.61

In [10]:
# Change all values in the room_type column to lowercase
room_types['room_type'] = room_types['room_type'].str.lower()

# Verify results
room_types

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,entire home/apt
1,3831,Cozy Entire Floor of Brownstone,entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,entire home/apt
...,...,...,...
25204,36425863,Lovely Privet Bedroom with Privet Restroom,private room
25205,36427429,No.2 with queen size bed,private room
25206,36438336,Seas The Moment,private room
25207,36442252,1B-1B apartment near by Metro,entire home/apt


In [11]:
# Get the frequencies of each room type
room_types['room_type'].value_counts()

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64

In [12]:
# Check the data type of the reviews df
reviews.dtypes

listing_id      int64
host_name      object
last_review    object
dtype: object

In [13]:
# Change the last_review column to pandas datetime data type
reviews['last_review'] = pd.to_datetime(reviews['last_review'])

# Verify the change
reviews.dtypes

listing_id              int64
host_name              object
last_review    datetime64[ns]
dtype: object

In [14]:
# Find the oldest review
first_review = reviews['last_review'].min()

# Find the most recent review
last_reviewed = reviews['last_review'].max()

print(f'The oldest review: {first_review}') 
print(f'The most recent review: {last_reviewed}')

The oldest review: 2019-01-01 00:00:00
The most recent review: 2019-07-09 00:00:00


In [15]:
# Merge prices with the room_types DataFame
rooms_and_prices = prices.merge(room_types, how = 'outer', on = 'listing_id')

# Merge rooms_and_prices with the reviews
airbnb_merged = rooms_and_prices.merge(reviews, how = 'outer', on = 'listing_id')

# Remove missing observations from airbnb_merged
airbnb_merged.dropna(inplace=True)

# Check for duplicates
print(airbnb_merged.duplicated().sum())

# Display the results
airbnb_merged.head()

0


,listing_id,price,nbhood_full,price_per_month,description,room_type,host_name,last_review
0,2595,225.0,"Manhattan, Midtown",6843.750000,Skylit Midtown Castle,entire home/apt,Jennifer,2019-05-21
1,3831,89.0,"Brooklyn, Clinton Hill",2707.083333,Cozy Entire Floor of Brownstone,entire home/apt,LisaRoxanne,2019-07-05
2,5099,200.0,"Manhattan, Murray Hill",6083.333333,Large Cozy 1 BR Apartment In Midtown East,entire home/apt,Chris,2019-06-22
3,5178,79.0,"Manhattan, Hell's Kitchen",2402.916667,Large Furnished Room Near B'way,private room,Shunichi,2019-06-24
4,5238,150.0,"Manhattan, Chinatown",4562.500000,Cute & Cozy Lower East Side 1 bdrm,entire home/apt,Ben,2019-06-09


In [18]:
# Create a new column in airbnb_merged called 'borough'
airbnb_merged['borough'] = airbnb_merged['nbhood_full'].str.partition(',')[0]

# Create a DataFrame 'boroughs' by grouping by grouping by 'borough' and calculating summary statistics
boroughs = airbnb_merged.groupby('borough')['price'].agg(['sum', 'mean',  'median', 'count'])

# Round all values in 'boroughs' DataFrame to nearest 2 decimal place and sort by mean in descending order
boroughs = boroughs.round(2).sort_values('mean', ascending=False)

# Display DataFrame
boroughs

,sum,mean,median,count
borough,,,,
Manhattan,1898417.0,184.04,149.0,10315
Brooklyn,1275250.0,122.02,95.0,10451
Queens,320715.0,92.83,70.0,3455
Staten Island,22974.0,86.04,71.0,267
Bronx,55156.0,79.25,65.0,696
